# Multi-agent Systems and Agentic Patterns with CrewAI

## Introduction

**CrewAI** is a powerful framework for orchestrating role-playing AI agents that work together to accomplish complex tasks. It provides a simple yet flexible API for creating multi-agent systems where each agent has specific roles, goals, and capabilities.

In this tutorial, we'll explore five fundamental agentic patterns using CrewAI with the **gpt-4.1-mini** model via the OpenAI API.

## 1. Setup & Installation

First, let's install the required libraries.

In [3]:
# Install required packages
# CrewAI: The main framework for multi-agent orchestration
# crewai-tools: Additional tools for agents

!pip install -q crewai crewai-tools openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.3/89.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.4/814.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.9/763.9 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.4/723.4 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Configure OpenAI API Key

You'll need an OpenAI API key to use their LLM services. Get one at: https://platform.openai.com/api-keys

In [4]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

## 2. Model Configuration

Let's create a reusable LLM configuration using OpenAI's gpt-4o-mini model.

In [5]:
from crewai import LLM

llm = LLM(
    model="gpt-4.1-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL")
)

print("✅ Model configured successfully!")
print(f"📋 Using: gpt-4.1-mini")
print(f"📋 Base URL: {os.environ.get('OPENAI_BASE_URL')}")

✅ Model configured successfully!
📋 Using: gpt-4.1-mini
📋 Base URL: https://ds-ai-internship.openai.azure.com/openai/v1/


## 3. Pattern A: Prompt Chaining

**Concept**: In prompt chaining, the output of one agent becomes the input for the next agent, creating a sequential pipeline of processing.

**Use Case**: When a task naturally breaks into sequential steps where each step depends on the previous one.

**Example**: Research → Summarization

In [6]:
from crewai import Agent, Task, Crew

# Agent A: Research Agent
# This agent gathers detailed information on a topic
research_agent = Agent(
    role="Research Specialist",
    goal="Gather comprehensive information about {topic}",
    backstory="You are an expert researcher with deep knowledge across multiple domains. "
              "You excel at finding relevant facts and insights.",
    llm=llm,
    verbose=True
)

# Agent B: Summary Agent
# This agent takes the research output and creates a concise summary
summary_agent = Agent(
    role="Content Summarizer",
    goal="Create a clear and concise summary of the research findings",
    backstory="You are a skilled writer who excels at distilling complex information "
              "into easily digestible summaries.",
    llm=llm,
    verbose=True
)

# Task 1: Research Task (Agent A)
# Output will be passed to the next task
research_task = Task(
    description="Research and gather detailed information about {topic}. "
                "Include key facts, recent developments, and important context.",
    agent=research_agent,
    expected_output="A comprehensive research document with key facts and insights"
)

# Task 2: Summary Task (Agent B)
# This task receives the output from research_task as context
summary_task = Task(
    description="Take the research findings and create a concise 3-paragraph summary. "
                "Focus on the most important points.",
    agent=summary_agent,
    expected_output="A clear 3-paragraph summary of the research findings",
    context=[research_task]  # This creates the chain: research_task output → summary_task input
)

# Create the crew with sequential process (default)
chain_crew = Crew(
    agents=[research_agent, summary_agent],
    tasks=[research_task, summary_task],
    verbose=True
)

# Execute the chain
print("=" * 60)
print("PATTERN A: PROMPT CHAINING EXAMPLE")
print("=" * 60)

result = chain_crew.kickoff(inputs={"topic": "Quantum Computing"})
print("\n" + "=" * 60)
print("FINAL RESULT:")
print("=" * 60)
print(result)

PATTERN A: PROMPT CHAINING EXAMPLE


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  aaaf15f8-3b5e-49b5-831c-619c656b0bfc                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Research and gather detailed information about Quantum Computing. Include key facts, recent              │
│  developments, and important context.                                                                           │
│  ID: 314f7bb2-2c23-4dfb-87b2-1956ece4de5c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Task: Research and gather detailed information about Quantum Computing. Include key facts, recent              │
│  developments, and important context.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Research Specialist                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Comprehensive Research Document on Quantum Computing**                                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Introduction to Quantum Computing                                                                       │
│                                                                                                                 │
│  Quantum computing is an emerging field of computing based on the principles of quantum mechanics, which is     │
│  the fundamental theory in physics describing nature at the smallest scales, such as atoms and subatomic        │
│  particles. Unlike classical computing, which uses bits as the smallest unit of information (0 or 1), quantum   │
│  computing uses qubits (quantum bits), which can exist in superpositions of 0 and 1 simultaneously due to the   │
│  phenomena of quantum superposition and entanglement.                                                           │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 2. Key Concepts in Quantum Computing                                                                       │
│                                                                                                                 │
│  #### 2.1 Qubits                                                                                                │
│  - **Definition:** The basic unit of quantum information.                                                       │
│  - **Superposition:** Qubits can be in a state that is a combination of |0⟩ and |1⟩ simultaneously.             │
│  - **Entanglement:** A quantum phenomenon where qubits become interconnected such that the state of one qubit   │
│  directly correlates with the state of another, no matter the distance between them.                            │
│  - **Decoherence:** The loss of quantum coherence where quantum information is lost due to interaction with     │
│  the environment, a major challenge in maintaining stable qubits.                                               │
│                                                                                                                 │
│  #### 2.2 Quantum Gates and Circuits                                                                            │
│  Quantum information is manipulated using quantum gates (akin to logic gates in classical computers), which     │
│  operate on qubits through unitary transformations. Quantum circuits are sequences of quantum gates designed    │
│  to perform specific algorithms.                                                                                │
│                                                                                                                 │
│  #### 2.3 Quantum Algorithms                           

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Summarizer                                                                                      │
│                                                                                                                 │
│  Task: Take the research findings and create a concise 3-paragraph summary. Focus on the most important         │
│  points.                                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Summarizer                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Quantum computing is a transformative paradigm based on quantum mechanics principles, utilizing qubits that    │
│  exploit superposition and entanglement to encode and process information in ways fundamentally different from  │
│  classical bits. Qubits can exist simultaneously in multiple states and become interlinked over distance        │
│  through entanglement, enabling exponential gains in computational power for select problems. Quantum gates     │
│  manipulate qubits via unitary operations forming quantum circuits, and specialized algorithms such as Shor’s   │
│  for factoring and Grover’s for database search demonstrate significant speedups over classical counterparts.   │
│  Different models of quantum computing exist, including gate-based systems, quantum annealers for               │
│  optimization, and topological approaches promising robust qubit encoding.                                      │
│                                                                                                                 │
│  Significant progress has been made in hardware development, with superconducting circuits, trapped ions,       │
│  photonic qubits, and spin-based implementations each offering unique advantages and challenges related to      │
│  coherence times, error rates, and scalability. Recent breakthroughs include Google’s 2019 demonstration of     │
│  quantum supremacy and ongoing industry efforts to scale to thousands of qubits, exemplified by IBM’s roadmap   │
│  towards the 1,000+ qubit “Quantum Condor” processor. Advances in quantum error correction and hybrid           │
│  quantum-classical algorithms such as the Variational Quantum Eigensolver and Quantum Approximate Optimization  │
│  Algorithm show promise for near-term practical applications. Additionally, increased government funding and a  │
│  growing software ecosystem are accelerating research and the commercialization of quantum technologies.        │
│                                                                                                                 │
│  The most compelling applications of quantum computing span cryptography, optimization, quantum simulation,     │
│  machine learning, and material science, potentially revolutionizing fields dependent on complex computations.  │
│  However, substantial challenges remain, including managing high error rates, improving qubit coherence,        │
│  scaling up physical qubits, and developing fault-tolerant architectures and algorithms. While the technology   │
│  is currently in an experimental phase with noisy intermediate-scale quantum devices, the near-term outlook     │
│  anticipates incremental advances and useful simulations. Over the next decade or more, the development of      │
│  universal fault-tolerant quantum computers could enable computational capabilities unattainable by classical   │
│  means, paving the way for transformative impacts in science, industry, and security.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Take the research findings and create a concise 3-paragraph summary. Focus on the most important points.       │
│  Agent:                                                                                                         │
│  Content Summarizer                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL RESULT:
Quantum computing is a transformative paradigm based on quantum mechanics principles, utilizing qubits that exploit superposition and entanglement to encode and process information in ways fundamentally different from classical bits. Qubits can exist simultaneously in multiple states and become interlinked over distance through entanglement, enabling exponential gains in computational power for select problems. Quantum gates manipulate qubits via unitary operations forming quantum circuits, and specialized algorithms such as Shor’s for factoring and Grover’s for database search demonstrate significant speedups over classical counterparts. Different models of quantum computing exist, including gate-based systems, quantum annealers for optimization, and topological approaches promising robust qubit encoding.

Significant progress has been made in hardware development, with superconducting circuits, trapped ions, photonic qubits, and spin-based implementations each offering

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 4. Pattern B: Routing

**Concept**: A router agent analyzes incoming requests and directs them to the appropriate specialized agent based on content or intent.

**Use Case**: When you have multiple specialized agents and need intelligent request distribution.

**Example**: Customer Service Router → Specialized Support Agents

In [7]:
# Specialized Agent 1: Technical Support
technical_agent = Agent(
    role="Technical Support Specialist",
    goal="Resolve technical issues and provide troubleshooting guidance",
    backstory="You are a technical expert with deep knowledge of software, hardware, "
              "and IT systems. You excel at diagnosing and solving technical problems.",
    llm=llm,
    verbose=True
)

# Specialized Agent 2: Sales Agent
sales_agent = Agent(
    role="Sales Representative",
    goal="Provide product information and help customers make purchase decisions",
    backstory="You are a knowledgeable sales professional who understands customer needs "
              "and can recommend the right products and services.",
    llm=llm,
    verbose=True
)

# Specialized Agent 3: General Information Agent
general_agent = Agent(
    role="General Information Specialist",
    goal="Answer general questions about company policies, hours, and basic information",
    backstory="You are a friendly customer service representative with comprehensive "
              "knowledge of company policies and general information.",
    llm=llm,
    verbose=True
)

# Router Agent: Analyzes queries and routes to appropriate specialist
router_agent = Agent(
    role="Customer Service Router",
    goal="Analyze customer queries and determine which specialist should handle them",
    backstory="You are an intelligent routing system that can quickly categorize customer "
              "inquiries as: TECHNICAL (tech issues), SALES (product/pricing questions), "
              "or GENERAL (policies/hours/basic info).",
    llm=llm,
    verbose=True
)

# Router Task: Determines which specialist to use
routing_task = Task(
    description="Analyze this customer query: '{query}'\n\n"
                "Determine if this is a TECHNICAL, SALES, or GENERAL inquiry. "
                "Respond with ONLY one word: TECHNICAL, SALES, or GENERAL.",
    agent=router_agent,
    expected_output="One word: TECHNICAL, SALES, or GENERAL"
)

# Create tasks for each specialist
technical_task = Task(
    description="Handle this technical inquiry: '{query}'",
    agent=technical_agent,
    expected_output="Technical support response with solution steps"
)

sales_task = Task(
    description="Handle this sales inquiry: '{query}'",
    agent=sales_agent,
    expected_output="Sales response with product recommendations"
)

general_task = Task(
    description="Handle this general inquiry: '{query}'",
    agent=general_agent,
    expected_output="General information response"
)

# Function to route queries
def route_query(query):
    """Route a query to the appropriate specialist agent"""
    print("=" * 60)
    print(f"ROUTING QUERY: {query}")
    print("=" * 60)

    # First, determine the route
    route_crew = Crew(
        agents=[router_agent],
        tasks=[routing_task],
        verbose=True
    )

    route_result = route_crew.kickoff(inputs={"query": query})
    route = str(route_result).strip().upper()

    print(f"\n→ Route determined: {route}\n")

    # Then, execute the appropriate specialist task
    if "TECHNICAL" in route:
        specialist_crew = Crew(agents=[technical_agent], tasks=[technical_task], verbose=True)
    elif "SALES" in route:
        specialist_crew = Crew(agents=[sales_agent], tasks=[sales_task], verbose=True)
    else:
        specialist_crew = Crew(agents=[general_agent], tasks=[general_task], verbose=True)

    result = specialist_crew.kickoff(inputs={"query": query})

    print("\n" + "=" * 60)
    print("FINAL RESPONSE:")
    print("=" * 60)
    print(result)
    return result

# Test the routing pattern
print("\n\n" + "=" * 60)
print("PATTERN B: ROUTING EXAMPLE")
print("=" * 60 + "\n")

# Test with a technical query
route_query("My laptop won't connect to WiFi. How do I fix this?")



PATTERN B: ROUTING EXAMPLE

ROUTING QUERY: My laptop won't connect to WiFi. How do I fix this?


╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  7032520e-e4e3-4ee1-bc42-a90341a5f1f9                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Analyze this customer query: 'My laptop won't connect to WiFi. How do I fix this?'                       │
│                                                                                                                 │
│  Determine if this is a TECHNICAL, SALES, or GENERAL inquiry. Respond with ONLY one word: TECHNICAL, SALES, or  │
│  GENERAL.                                                                                                       │
│  ID: 7e270615-71ad-4779-97f7-515b9cc00874                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Service Router                                                                                 │
│                                                                                                                 │
│  Task: Analyze this customer query: 'My laptop won't connect to WiFi. How do I fix this?'                       │
│                                                                                                                 │
│  Determine if this is a TECHNICAL, SALES, or GENERAL inquiry. Respond with ONLY one word: TECHNICAL, SALES, or  │
│  GENERAL.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Service Router                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  TECHNICAL                                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Handle this technical inquiry: 'My laptop won't connect to WiFi. How do I fix this?'                           │
│  Agent:                                                                                                         │
│  Technical Support Specialist                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL RESPONSE:
I'm sorry to hear that your laptop won't connect to WiFi. Let's go through some troubleshooting steps to identify and fix the issue:

### Step 1: Check Basic Settings
1. **Ensure WiFi is turned on:**
   - On Windows: Click the WiFi icon in the system tray to see if WiFi is enabled.
   - On macOS: Click the WiFi icon in the menu bar and make sure WiFi is on.
2. **Make sure Airplane Mode is off:**
   - On Windows: Go to Settings > Network & Internet > Airplane Mode.
   - On macOS: Airplane mode isn’t typically used, but ensure no similar setting is enabled.
3. **Check the router and modem:**
   - Confirm your WiFi router is powered on and the internet connection is active.
   - Try connecting other devices (like your smartphone) to the WiFi to ensure the network is working.

### Step 2: Restart Devices
1. Restart your laptop.
2. Restart your WiFi router and modem by unplugging them from the power source for 30 seconds and plugging back in.

### Step 3: Reconnect to the N

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

CrewOutput(raw='I\'m sorry to hear that your laptop won\'t connect to WiFi. Let\'s go through some troubleshooting steps to identify and fix the issue:\n\n### Step 1: Check Basic Settings\n1. **Ensure WiFi is turned on:**\n   - On Windows: Click the WiFi icon in the system tray to see if WiFi is enabled.\n   - On macOS: Click the WiFi icon in the menu bar and make sure WiFi is on.\n2. **Make sure Airplane Mode is off:**\n   - On Windows: Go to Settings > Network & Internet > Airplane Mode.\n   - On macOS: Airplane mode isn’t typically used, but ensure no similar setting is enabled.\n3. **Check the router and modem:**\n   - Confirm your WiFi router is powered on and the internet connection is active.\n   - Try connecting other devices (like your smartphone) to the WiFi to ensure the network is working.\n\n### Step 2: Restart Devices\n1. Restart your laptop.\n2. Restart your WiFi router and modem by unplugging them from the power source for 30 seconds and plugging back in.\n\n### Step 3:

Would you like to view your execution traces? [y/N] (20s timeout): 



## 5. Pattern C: Parallelization

**Concept**: Multiple agents work simultaneously on different aspects of a problem, then their results are aggregated.

**Use Case**: When you have independent sub-tasks that can be executed concurrently to save time.

**Example**: Market Research + Competitor Analysis → Unified Report

In [8]:
from crewai import Process

# Agent 1: Market Research Agent (runs in parallel)
market_researcher = Agent(
    role="Market Research Analyst",
    goal="Analyze market trends, customer needs, and opportunities for {product}",
    backstory="You are a market research expert with years of experience in analyzing "
              "market dynamics, consumer behavior, and industry trends.",
    llm=llm,
    verbose=True
)

# Agent 2: Competitor Analysis Agent (runs in parallel)
competitor_analyst = Agent(
    role="Competitive Intelligence Analyst",
    goal="Analyze competitors, their strategies, and market positioning for {product}",
    backstory="You are a competitive intelligence specialist who excels at understanding "
              "competitor strengths, weaknesses, and market strategies.",
    llm=llm,
    verbose=True
)

# Agent 3: Aggregator Agent (runs after parallel tasks complete)
aggregator = Agent(
    role="Strategic Report Writer",
    goal="Combine market research and competitor analysis into a unified strategic report",
    backstory="You are a strategic analyst who excels at synthesizing multiple sources "
              "of information into actionable insights.",
    llm=llm,
    verbose=True
)

# Parallel Task 1: Market Research
market_task = Task(
    description="Conduct market research for {product}. Analyze:\n"
                "1. Market size and growth potential\n"
                "2. Target customer segments\n"
                "3. Key market trends\n"
                "4. Opportunities and challenges",
    agent=market_researcher,
    expected_output="Comprehensive market research report with key insights"
)

# Parallel Task 2: Competitor Analysis
competitor_task = Task(
    description="Analyze competitors in the {product} market. Include:\n"
                "1. Main competitors\n"
                "2. Their strengths and weaknesses\n"
                "3. Market share and positioning\n"
                "4. Competitive advantages",
    agent=competitor_analyst,
    expected_output="Detailed competitor analysis report"
)

# Aggregation Task: Combines results from both parallel tasks
aggregation_task = Task(
    description="Synthesize the market research and competitor analysis into a unified "
                "strategic report. Provide actionable recommendations.",
    agent=aggregator,
    expected_output="Unified strategic report with recommendations",
    context=[market_task, competitor_task]  # Waits for both tasks to complete
)

# Create crew with parallel process for first two tasks
# Note: CrewAI executes tasks sequentially by default, but we simulate parallelization
# by having independent tasks that don't depend on each other, then aggregating
parallel_crew = Crew(
    agents=[market_researcher, competitor_analyst, aggregator],
    tasks=[market_task, competitor_task, aggregation_task],
    verbose=True,
    process=Process.sequential  # Tasks 1&2 are independent; Task 3 aggregates
)

# Execute parallel pattern
print("=" * 60)
print("PATTERN C: PARALLELIZATION EXAMPLE")
print("=" * 60)
print("Note: Market research and competitor analysis run as independent tasks")
print("=" * 60 + "\n")

result = parallel_crew.kickoff(inputs={"product": "AI-powered fitness app"})

print("\n" + "=" * 60)
print("AGGREGATED RESULT:")
print("=" * 60)
print(result)

PATTERN C: PARALLELIZATION EXAMPLE
Note: Market research and competitor analysis run as independent tasks



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  cb56d0ba-9195-4561-8b5d-7ddc7774123f                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Conduct market research for AI-powered fitness app. Analyze:                                             │
│  1. Market size and growth potential                                                                            │
│  2. Target customer segments                                                                                    │
│  3. Key market trends                                                                                           │
│  4. Opportunities and challenges                                                                                │
│  ID: 9993da71-fb5b-4df5-bab7-31a0ce47c353                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Conduct market research for AI-powered fitness app. Analyze:                                             │
│  1. Market size and growth potential                                                                            │
│  2. Target customer segments                                                                                    │
│  3. Key market trends                                                                                           │
│  4. Opportunities and challenges                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Market Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Market Research Report: AI-Powered Fitness App                                                                 │
│                                                                                                                 │
│  1. Market Size and Growth Potential                                                                            │
│                                                                                                                 │
│  The global fitness app market was valued at approximately USD 4.4 billion in 2023 and is projected to grow at  │
│  a compound annual growth rate (CAGR) of around 21% from 2023 to 2030. The surge is driven by increasing        │
│  consumer health awareness, widespread smartphone penetration, and advancements in AI and machine learning      │
│  technologies that enhance personalized fitness experiences.                                                    │
│                                                                                                                 │
│  Specifically, AI-powered fitness apps represent a rapidly expanding sub-segment due to their ability to        │
│  provide customized workout plans, real-time feedback, and adaptive training regimes based on user data and     │
│  preferences. Industry forecasts predict the AI fitness app market alone could grow to USD 2.5 billion by       │
│  2030, reflecting heightened demand for smart, data-driven health solutions.                                    │
│                                                                                                                 │
│  Key drivers fueling market expansion include:                                                                  │
│                                                                                                                 │
│  - Rising adoption of wearable devices that integrate with fitness apps                                         │
│                                                                                                                 │
│  - Growing preference for at-home and on-demand fitness solutions post-COVID-19                                 │
│                                                                                                                 │
│  - Increasing focus on personalized health and wellness leveraging AI insights                                  │
│                                                                                                                 │
│  - Expansion of the digital health and telefitness ecosystem globally                                           │
│                                                                                                                 │
│  2. Target Customer Segments                                                                                    │
│                                                                                                                 │
│  The customer base for AI-powered fitness apps covers diverse demographic and psychographic segments:           │
│                                                                                                                 │
│  a) Fitness Enthusiasts and Athletes                                                                            │
│  - Often aged 18-40, tech-savvy, motivated by performan

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Strategic Report Writer                                                                                 │
│                                                                                                                 │
│  Task: Synthesize the market research and competitor analysis into a unified strategic report. Provide          │
│  actionable recommendations.                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Strategic Report Writer                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Unified Strategic Report: AI-Powered Fitness App Market**                                                    │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### Executive Summary                                                                                          │
│                                                                                                                 │
│  The AI-powered fitness app market is a high-growth, dynamic sector projected to expand at a CAGR of            │
│  approximately 21% through 2030, reaching an estimated USD 2.5 billion submarket within the broader USD 4.4     │
│  billion fitness app ecosystem. This growth is driven by technological advancements in AI and machine           │
│  learning, rising consumer demand for personalized and data-driven fitness solutions, increased adoption of     │
│  wearable technology, and growing preferences for flexible, at-home fitness alternatives.                       │
│                                                                                                                 │
│  Competition is intense, featuring well-established global brands and innovative startups leveraging AI to      │
│  deliver differentiated user experiences through personalized workouts, real-time feedback, social engagement,  │
│  and holistic wellness integration. Market leaders like Peloton and Nike leverage their brand equity and        │
│  expansive ecosystems, while niche players such as Freeletics, Fitbod, and Aaptiv excel at focused offerings    │
│  through advanced AI algorithms and novel engagement modes.                                                     │
│                                                                                                                 │
│  Successful market participation hinges on balancing superior AI personalization, seamless integration with     │
│  wearables/IoT, comprehensive wellness features, and addressing challenges including data privacy, user         │
│  retention, and ethical AI use. Emerging markets and corporate wellness programs present promising growth       │
│  opportunities.                                                                                                 │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### 1. Market Landscape                                                                                        │
│                                                                                                                 │
│  **Market Size & Growth Drivers**                                                                               │
│                                                                                                                 │
│  - AI fitness apps represent a fast-growing segment wit

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Synthesize the market research and competitor analysis into a unified strategic report. Provide actionable     │
│  recommendations.                                                                                               │
│  Agent:                                                                                                         │
│  Strategic Report Writer                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


AGGREGATED RESULT:
**Unified Strategic Report: AI-Powered Fitness App Market**

---

### Executive Summary

The AI-powered fitness app market is a high-growth, dynamic sector projected to expand at a CAGR of approximately 21% through 2030, reaching an estimated USD 2.5 billion submarket within the broader USD 4.4 billion fitness app ecosystem. This growth is driven by technological advancements in AI and machine learning, rising consumer demand for personalized and data-driven fitness solutions, increased adoption of wearable technology, and growing preferences for flexible, at-home fitness alternatives.

Competition is intense, featuring well-established global brands and innovative startups leveraging AI to deliver differentiated user experiences through personalized workouts, real-time feedback, social engagement, and holistic wellness integration. Market leaders like Peloton and Nike leverage their brand equity and expansive ecosystems, while niche players such as Freeletics, Fitb

## 6. Pattern D: Orchestrator-Worker

**Concept**: A manager/orchestrator agent breaks down complex tasks into subtasks and delegates them to worker agents sequentially.

**Use Case**: When a complex task needs to be decomposed into smaller, manageable steps that must be executed in order.

**Example**: Manager Agent → Worker Agent (Research → Outline → Draft → Edit)

In [9]:
# Manager Agent: Orchestrates the overall process
manager_agent = Agent(
    role="Project Manager",
    goal="Break down the task of writing a blog post about {topic} into clear subtasks",
    backstory="You are an experienced project manager who excels at decomposing complex "
              "tasks into manageable steps and ensuring quality execution.",
    llm=llm,
    verbose=True
)

# Worker Agent: Executes the subtasks
worker_agent = Agent(
    role="Content Worker",
    goal="Execute assigned subtasks efficiently and effectively",
    backstory="You are a versatile content professional capable of research, writing, "
              "and editing. You follow instructions precisely and deliver quality work.",
    llm=llm,
    verbose=True
)

# Step 1: Manager plans the work
planning_task = Task(
    description="Create a plan for writing a blog post about {topic}. "
                "List the key steps needed (research, outline, draft, edit).",
    agent=manager_agent,
    expected_output="A clear plan with 4 steps for creating the blog post"
)

# Step 2: Worker conducts research
research_step = Task(
    description="SUBTASK 1: Research {topic}. Gather key facts, statistics, and insights "
                "that would be valuable for a blog post.",
    agent=worker_agent,
    expected_output="Research findings with key facts and insights",
    context=[planning_task]
)

# Step 3: Worker creates outline
outline_step = Task(
    description="SUBTASK 2: Based on the research, create a detailed outline for a blog post "
                "about {topic}. Include main sections and key points.",
    agent=worker_agent,
    expected_output="Detailed blog post outline with sections and key points",
    context=[research_step]
)

# Step 4: Worker writes draft
draft_step = Task(
    description="SUBTASK 3: Using the outline, write a complete draft of the blog post. "
                "Make it engaging and informative (approximately 300 words).",
    agent=worker_agent,
    expected_output="Complete blog post draft",
    context=[outline_step]
)

# Step 5: Worker edits and finalizes
edit_step = Task(
    description="SUBTASK 4: Review and edit the draft. Improve clarity, fix any issues, "
                "and ensure the blog post is polished and ready for publication.",
    agent=worker_agent,
    expected_output="Final, polished blog post ready for publication",
    context=[draft_step]
)

# Create the orchestrator-worker crew
orchestrator_crew = Crew(
    agents=[manager_agent, worker_agent],
    tasks=[planning_task, research_step, outline_step, draft_step, edit_step],
    verbose=True,
    process=Process.sequential
)

# Execute the orchestrator-worker pattern
print("=" * 60)
print("PATTERN D: ORCHESTRATOR-WORKER EXAMPLE")
print("=" * 60)
print("Manager breaks down task → Worker executes each step")
print("=" * 60 + "\n")

result = orchestrator_crew.kickoff(inputs={"topic": "The Future of Renewable Energy"})

print("\n" + "=" * 60)
print("FINAL BLOG POST:")
print("=" * 60)
print(result)

PATTERN D: ORCHESTRATOR-WORKER EXAMPLE
Manager breaks down task → Worker executes each step



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  f9179f27-8260-40e6-92cf-db256f06fbb3                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Create a plan for writing a blog post about The Future of Renewable Energy. List the key steps needed    │
│  (research, outline, draft, edit).                                                                              │
│  ID: 45b3a584-4b5b-48e8-9034-a93e1045dba6                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Manager                                                                                         │
│                                                                                                                 │
│  Task: Create a plan for writing a blog post about The Future of Renewable Energy. List the key steps needed    │
│  (research, outline, draft, edit).                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Project Manager                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Plan for Writing a Blog Post About The Future of Renewable Energy                                              │
│                                                                                                                 │
│  Step 1: Research                                                                                               │
│  - Gather recent and credible information on renewable energy technologies, trends, and forecasts.              │
│  - Identify key topics such as solar, wind, bioenergy, policy changes, innovations, and global adoption rates.  │
│  - Collect data from reputable sources like scientific journals, industry reports, government publications,     │
│  and expert opinions.                                                                                           │
│  - Take notes and organize findings to support the blog’s key points.                                           │
│                                                                                                                 │
│  Step 2: Outline                                                                                                │
│  - Define the blog post structure with an engaging introduction, body sections, and a strong conclusion.        │
│  - Break down the body into clear subsections such as current state, emerging technologies, challenges, and     │
│  future outlook.                                                                                                │
│  - Decide on key messages and supporting evidence for each section.                                             │
│  - Create bullet points or headings to serve as a roadmap for writing the draft.                                │
│                                                                                                                 │
│  Step 3: Draft                                                                                                  │
│  - Write the initial version of the blog post following the outline.                                            │
│  - Ensure the content is clear, engaging, and accessible to the target audience.                                │
│  - Incorporate data, examples, and quotes gathered during research.                                             │
│  - Maintain a logical flow between sections and use subheadings for readability.                                │
│                                                                                                                 │
│  Step 4: Edit                                                                                                   │
│  - Review the draft for clarity, coherence, and grammar.                                                        │
│  - Check for factual accuracy and update any information as needed.                                             │
│  - Refine language to enhance engagement and tone appropriate to the audience.                                  │
│  - Format the post with visuals if applicable and ensure it meets the publication’s style guidelines.           │
│  - Perform a final proofreading before submission or publishing.                                                │
│                                                                                                                 │
│  This four-step plan provides a clear roadmap to create

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Worker                                                                                          │
│                                                                                                                 │
│  Task: SUBTASK 4: Review and edit the draft. Improve clarity, fix any issues, and ensure the blog post is       │
│  polished and ready for publication.                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Worker                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The Future of Renewable Energy: Powering a Sustainable Tomorrow                                                │
│                                                                                                                 │
│  The world is rapidly transitioning to renewable energy—a critical step in combating climate change and         │
│  securing a sustainable future. By 2023, global renewable capacity reached approximately 3,000 gigawatts, a     │
│  10% increase from the previous year. Solar and wind power now make up over 70% of all renewable                │
│  installations. Leading this transformation are countries like China, the United States, and members of the     │
│  European Union, driving innovation and expanding clean energy access worldwide.                                │
│                                                                                                                 │
│  Emerging technologies promise to accelerate this energy revolution even further. Advanced solar cells using    │
│  perovskite materials are breaking efficiency records, while tandem solar cells that combine silicon and        │
│  perovskite layers aim to exceed traditional efficiency limits. Floating offshore wind farms are unlocking      │
│  vast new areas far from shorelines by harnessing stronger winds at sea, with capacity expected to surpass 10   │
│  gigawatts by 2030. Meanwhile, green hydrogen—a clean fuel produced by splitting water using renewable          │
│  electricity—is becoming vital for decarbonizing hard-to-electrify sectors such as steelmaking and aviation.    │
│  Energy storage technologies, particularly lithium-ion and next-generation solid-state batteries, are growing   │
│  rapidly to address the intermittent nature of renewables, with the global storage market projected to exceed   │
│  $400 billion by 2030. Additionally, digitalization, artificial intelligence, and blockchain innovations are    │
│  optimizing grid management and energy trading.                                                                 │
│                                                                                                                 │
│  Policy momentum is another key driver. More than 130 countries have pledged net-zero emissions by              │
│  mid-century, supported by landmark initiatives such as the U.S. Inflation Reduction Act and the European       │
│  Green Deal. Rapid renewable adoption in developing regions is enabled by falling technology costs and          │
│  international financing. However, challenges remain: integrating variable renewables into grids requires       │
│  infrastructure upgrades, and the reliance on critical minerals raises geopolitical and environmental           │
│  concerns. Ensuring equitable energy access and managing social and ecological impacts remain paramount         │
│  priorities.                                                                                                    │
│                                                                                                                 │
│  Looking ahead, the outlook is promising. By 2050, renewable capacity could soar to 8,500 gigawatts, supplying  │
│  nearly 90% of global electricity. Renewables are expected to become the cheapest source of power in most       │
│  regions by 2030, enabling deep decarbonization across 

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  SUBTASK 4: Review and edit the draft. Improve clarity, fix any issues, and ensure the blog post is polished    │
│  and ready for publication.                                                                                     │
│  Agent:                                                                                                         │
│  Content Worker                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL BLOG POST:
The Future of Renewable Energy: Powering a Sustainable Tomorrow

The world is rapidly transitioning to renewable energy—a critical step in combating climate change and securing a sustainable future. By 2023, global renewable capacity reached approximately 3,000 gigawatts, a 10% increase from the previous year. Solar and wind power now make up over 70% of all renewable installations. Leading this transformation are countries like China, the United States, and members of the European Union, driving innovation and expanding clean energy access worldwide.

Emerging technologies promise to accelerate this energy revolution even further. Advanced solar cells using perovskite materials are breaking efficiency records, while tandem solar cells that combine silicon and perovskite layers aim to exceed traditional efficiency limits. Floating offshore wind farms are unlocking vast new areas far from shorelines by harnessing stronger winds at sea, with capacity expected to surpass

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 7. Pattern E: Evaluator-Optimizer

**Concept**: An iterative feedback loop where one agent generates content, another evaluates it, and the generator improves based on feedback.

**Use Case**: When output quality is critical and needs iterative refinement.

**Example**: Content Generator → Evaluator → Improved Generator (Feedback Loop)

In [10]:
# Generator Agent: Creates initial content
generator_agent = Agent(
    role="Content Generator",
    goal="Create compelling marketing copy for {product}",
    backstory="You are a creative copywriter who generates engaging marketing content. "
              "You're open to feedback and continuously improve your work.",
    llm=llm,
    verbose=True
)

# Evaluator Agent: Critiques the content
evaluator_agent = Agent(
    role="Marketing Copy Evaluator",
    goal="Evaluate marketing copy and provide constructive feedback",
    backstory="You are a seasoned marketing director with a keen eye for effective copy. "
              "You evaluate content based on: clarity, persuasiveness, emotional appeal, "
              "and call-to-action strength.",
    llm=llm,
    verbose=True
)

# Optimizer Agent: Improves based on feedback
optimizer_agent = Agent(
    role="Content Optimizer",
    goal="Improve marketing copy based on evaluator feedback",
    backstory="You are a skilled editor who excels at refining content based on feedback. "
              "You make targeted improvements while maintaining the original creative vision.",
    llm=llm,
    verbose=True
)

# Iteration 1: Generate initial content
generation_task = Task(
    description="Write a compelling 100-word marketing copy for {product}. "
                "Include a strong headline and a clear call-to-action.",
    agent=generator_agent,
    expected_output="Initial marketing copy with headline and call-to-action"
)

# Iteration 2: Evaluate the content
evaluation_task = Task(
    description="Evaluate the marketing copy. Rate it on:\n"
                "1. Clarity (1-10)\n"
                "2. Persuasiveness (1-10)\n"
                "3. Emotional Appeal (1-10)\n"
                "4. Call-to-Action Strength (1-10)\n\n"
                "Provide specific suggestions for improvement.",
    agent=evaluator_agent,
    expected_output="Detailed evaluation with ratings and improvement suggestions",
    context=[generation_task]
)

# Iteration 3: Optimize based on feedback
optimization_task = Task(
    description="Improve the marketing copy based on the evaluator's feedback. "
                "Address each suggestion and create a refined version.",
    agent=optimizer_agent,
    expected_output="Improved marketing copy incorporating feedback",
    context=[generation_task, evaluation_task]
)

# Iteration 4: Re-evaluate the improved version
final_evaluation_task = Task(
    description="Re-evaluate the improved marketing copy. Compare it to the original "
                "and confirm the improvements. Provide final ratings.",
    agent=evaluator_agent,
    expected_output="Final evaluation confirming improvements",
    context=[optimization_task]
)

# Create the evaluator-optimizer crew
feedback_loop_crew = Crew(
    agents=[generator_agent, evaluator_agent, optimizer_agent],
    tasks=[generation_task, evaluation_task, optimization_task, final_evaluation_task],
    verbose=True,
    process=Process.sequential
)

# Execute the feedback loop
print("=" * 60)
print("PATTERN E: EVALUATOR-OPTIMIZER EXAMPLE")
print("=" * 60)
print("Generate → Evaluate → Optimize → Re-evaluate")
print("=" * 60 + "\n")

result = feedback_loop_crew.kickoff(
    inputs={"product": "An AI-powered personal finance app that helps users save money"}
)

print("\n" + "=" * 60)
print("FINAL RESULT (After Feedback Loop):")
print("=" * 60)
print(result)

PATTERN E: EVALUATOR-OPTIMIZER EXAMPLE
Generate → Evaluate → Optimize → Re-evaluate



╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  303ff140-6e89-4aed-b68e-6ec99a0f98d7                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: Write a compelling 100-word marketing copy for An AI-powered personal finance app that helps users save  │
│  money. Include a strong headline and a clear call-to-action.                                                   │
│  ID: d52abf50-a933-46e3-958b-378a5b05f0c1                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Generator                                                                                       │
│                                                                                                                 │
│  Task: Write a compelling 100-word marketing copy for An AI-powered personal finance app that helps users save  │
│  money. Include a strong headline and a clear call-to-action.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Generator                                                                                       │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Save Smarter, Live Better with MoneyMate!                                                                      │
│                                                                                                                 │
│  Unlock the power of AI to take control of your finances like never before. MoneyMate analyzes your spending    │
│  habits, identifies hidden savings, and delivers personalized tips to boost your bank balance effortlessly.     │
│  Whether it's cutting unnecessary expenses or optimizing your budget, our smart app is your pocket-sized        │
│  financial advisor—available 24/7. Join thousands who have already transformed their money management and       │
│  started saving more every day. Don’t just spend—spend wisely and watch your savings grow!                      │
│                                                                                                                 │
│  Download MoneyMate now and start your journey to financial freedom today!                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Marketing Copy Evaluator                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Save Smarter, Live Better with MoneyMate!                                                                      │
│                                                                                                                 │
│  Feeling overwhelmed or uncertain about your finances? You’re not alone—and MoneyMate is here to help. Harness  │
│  the power of AI to seize control of your money like never before. MoneyMate dives deep into your spending      │
│  patterns, effortlessly uncovering hidden savings and delivering personalized, actionable tips that help grow   │
│  your bank balance with confidence. Whether you’re trimming unnecessary expenses or perfecting your budget,     │
│  our smart app is your trusted 24/7 financial advisor in your pocket.                                           │
│                                                                                                                 │
│  Join thousands who’ve already transformed their money management—reducing financial stress and unlocking       │
│  lasting peace of mind every day. Don’t let another dollar slip through the cracks. Start saving smarter        │
│  today!                                                                                                         │
│                                                                                                                 │
│  Download MoneyMate now and get your personalized savings plan in minutes—risk free! Take the first step        │
│  toward true financial freedom and watch your savings soar!                                                     │
│                                                                                                                 │
│  “Since using MoneyMate, I’ve saved hundreds without changing my lifestyle. It’s like having a financial coach  │
│  with me 24/7.” – Jamie L.                                                                                      │
│                                                                                                                 │
│  Make the switch today—because your money deserves the smartest strategy.                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  Final Evaluation and Ratings:                                                                                  │
│                                                                                                                 │
│  1. Clarity: 9/10                                                                                               │
│  The improved copy maintains excellent clarity with detailed yet understandable language. The addition of       │
│  empathetic opening lines helps orient the reader better. The message is straightforward and accessible.        │
│                                                                                                                 │
│  2. Persuasiveness: 9/10                                                                                        │
│  Persuasiveness is significantly enhanced by the inclus

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Re-evaluate the improved marketing copy. Compare it to the original and confirm the improvements. Provide      │
│  final ratings.                                                                                                 │
│  Agent:                                                                                                         │
│  Marketing Copy Evaluator                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL RESULT (After Feedback Loop):
Save Smarter, Live Better with MoneyMate!

Feeling overwhelmed or uncertain about your finances? You’re not alone—and MoneyMate is here to help. Harness the power of AI to seize control of your money like never before. MoneyMate dives deep into your spending patterns, effortlessly uncovering hidden savings and delivering personalized, actionable tips that help grow your bank balance with confidence. Whether you’re trimming unnecessary expenses or perfecting your budget, our smart app is your trusted 24/7 financial advisor in your pocket.

Join thousands who’ve already transformed their money management—reducing financial stress and unlocking lasting peace of mind every day. Don’t let another dollar slip through the cracks. Start saving smarter today!

Download MoneyMate now and get your personalized savings plan in minutes—risk free! Take the first step toward true financial freedom and watch your savings soar!

“Since using MoneyMate, I’ve saved hu

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

## 8. Conclusion: When to Use Each Pattern

Now that we've explored the five fundamental agentic patterns, let's summarize when to use each one:

### Pattern Selection Guide

| Pattern | Best Used When | Key Benefits | Example Use Cases |
|---------|---------------|--------------|-------------------|
| **Prompt Chaining** | Tasks naturally flow in sequence, each step depends on the previous | Simple, predictable, easy to debug | Data pipeline (extract → transform → load), Research → Summary |
| **Routing** | You have multiple specialized agents and need dynamic selection | Efficient resource use, specialized expertise | Customer service routing, Intent classification, Dynamic tool selection |
| **Parallelization** | Independent subtasks can run simultaneously | Time efficiency, concurrent processing | Market research + competitor analysis, Multiple data source aggregation |
| **Orchestrator-Worker** | Complex tasks need decomposition into managed steps | Clear structure, scalable delegation | Project planning, Multi-step content creation, Complex workflows |
| **Evaluator-Optimizer** | Quality is critical and iterative improvement needed | High-quality output, continuous refinement | Content creation, Code review, Strategic planning |

### Combining Patterns

These patterns aren't mutually exclusive! In production systems, you might combine them:
- **Routing + Parallelization**: Route queries to multiple specialists who work in parallel
- **Orchestrator-Worker + Evaluator-Optimizer**: Orchestrate a process with quality checks at each step
- **Prompt Chaining + Routing**: Chain multiple steps where each step routes to different specialists

### Key Takeaways

1. **Start Simple**: Begin with Prompt Chaining for linear workflows
2. **Add Intelligence**: Use Routing when you need dynamic decision-making
3. **Scale Performance**: Apply Parallelization for time-critical operations
4. **Manage Complexity**: Use Orchestrator-Worker for complex, multi-step tasks
5. **Ensure Quality**: Implement Evaluator-Optimizer when output quality is paramount

### Next Steps

- Experiment with each pattern using your own use cases
- Try combining patterns for more sophisticated workflows
- Explore CrewAI's additional features: custom tools, memory, and advanced configurations
- Monitor agent performance and iterate on your designs

Happy building with CrewAI! 🚀